<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup 

<h3>Scraping the Wikipedia page<h3>

In [2]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
r = requests.get(website_url) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
print("Scraped and got the data") 

Scraped and got the data


<h3>Wragling and Cleaning the data</h3>

In [6]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
selected = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            selected = False
            if columnNum == 1:
                if selected == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    selected = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

print('Data cleaned')

Data cleaned


<h3>Creating dataframe<h3>

In [8]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

neighborhood_df = pd.DataFrame(columns=column_names)

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighborhood_df = neighborhood_df.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)
    
neighborhood_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [9]:
neighborhood_df.shape

(210, 3)